# Basics of Tensor Network library

import numpy as np
import tensornetwork as tn

a = tn.Node(np.array([1.,2.,3.]))  # define a new node
b = tn.Node(np.array([4.,5.,6.]))  # define another new node

edge = a[0] ^ b[0]  # connect the nodes using 0th edge
c = tn.contract(edge)
print(c.tensor)

In [2]:
a = tn.Node(np.eye(2))  # This is a matrix, so has 2 dangling edges 
print(a[0].is_dangling(), a[1].is_dangling())

edge = a[0] ^ a[1]  # Connecting the edges
print(a[0] == a[1]) # After connection they represent the same edge

True True
True


## Axis naming

In [3]:
a = tn.Node(np.eye(2), axis_names=['alpha', 'beta'])
edge = a['alpha'] ^ a['beta']  # a trace of the matrix
tn.contract(edge)

Node
(
name : '__unnamed_node__',
tensor : 
array(2.),
edges : 
[] 
)

## Simple contraction

Given a pair of tensors, there are two distinct ways one can contract them:

1. Contracting edges sequencially;
2. Contracting edges in parallel.

The second option works substantially faster, as shown in the code below.

In [4]:
from timeit import default_timer as timer

def one_edge_at_a_time(a, b):
  node1 = tn.Node(a)
  node2 = tn.Node(b)
  edge1 = node1[0] ^ node2[0]
  edge2 = node1[1] ^ node2[1]
  tn.contract(edge1)
  result = tn.contract(edge2)
  return result.tensor

def use_contract_between(a, b):
  node1 = tn.Node(a)
  node2 = tn.Node(b)
  node1[0] ^ node2[0]
  node1[1] ^ node2[1]
  # This is the same as 
  # tn.contract_between(node1, node2)
  result = node1 @ node2
  return result.tensor

a = np.ones((1000, 1000))
b = np.ones((1000, 1000))

start = timer()
print("Running one_edge_at_a_time")
one_edge_at_a_time(a, b)
print(timer() - start)

start = timer()
print("\nRunning use_cotract_between")
use_contract_between(a, b)
print(timer() - start)


Running one_edge_at_a_time
0.015898055999997496

Running use_cotract_between
0.0012846550000062962


## More complex contraction

In [5]:
a = tn.Node(np.ones((2, 2, 2)))
b = tn.Node(np.ones((2, 2, 2)))
c = tn.Node(np.ones((2, 2, 2)))
d = tn.Node(np.ones((2, 2, 2)))
a[0] ^ b[0]
a[1] ^ c[1]
a[2] ^ d[2]
b[1] ^ d[1]
b[2] ^ c[2]
c[0] ^ d[0]

nodes = tn.reachable(a)
result = tn.contractors.greedy(nodes)
print(result.tensor)

64.0


Alternatively, using the NCON package:

In [6]:
ones = np.ones((2, 2, 2))
tn.ncon([ones, ones, ones, ones], 
        [[1, 2, 4], 
         [1, 3, 5], 
         [2, 3, 6],
         [4, 5, 6]])

array(64.)